### Importing relevant libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline 
import matplotlib
from sklearn.model_selection import train_test_split

### Opening data file

In [ ]:
excel = pd.ExcelFile('Praga Poludnie_lokale_11_2020.xlsx')
df1 = pd.read_excel(excel, 'r.w. lokale 2018-2020')
df1.head()

### Removing irrelevant columns

In [ ]:
df2 = df1.drop(['Unnamed: 0', 'Rep.A', ' Dzielnica', 'Nr lok.', 'Izby2', 'Opis', 'Źródło danych', 'Data wpisu', 
                'Dodatkowe składniki', 'Cena dodatkowych składników [zł]', 'Cena jedn. [zł/m2]', 'RAZEM', 
                'Cena jedn. razem [zł/m2]', 'Rodzaj prawa do lokalu', 'Ulica', 'Nr bud.', 'Data transakcji','Rodzaj rynku'], axis=1)
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
df2.dtypes

### Converting columns to correct data types

In [ ]:
df2['Cena trans. [zł]'] = pd.to_numeric(df2['Cena trans. [zł]'], errors='coerce')
df2['Izby'] = pd.to_numeric(df2['Izby'], errors='coerce')
df2['Kond.'] = pd.to_numeric(df2['Kond.'], errors = 'coerce')
df2['Obręb'] = df2['Obręb'].astype('category')
df2 = df2.dropna(subset=['Cena trans. [zł]'])
df2.dtypes

### Filling missing values

In [ ]:
df2 = df2.fillna(df2.mean())
df2.isnull().sum()

### Assigning variables

In [ ]:
X = df2.drop('Cena trans. [zł]', axis=1)
y = df2['Cena trans. [zł]']

X.dtypes

### Putting dummies for categorical data

In [ ]:
X = pd.get_dummies(data=X, drop_first=True)
X.columns

In [ ]:
pd.plotting.register_matplotlib_converters()

### Assigning train and test data and running the model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

### Defining function to run prediction on new data

In [ ]:
def price_prediction(obreb: str, kond: int, pow_uzyt: float, izby: int):
    obreb_index = np.where(X.columns==obreb)[0][0]
    x = np.zeros(len(X.columns))
    x[0] = kond
    x[1] = pow_uzyt
    x[2] = izby
    if obreb_index >= 0:
        x[obreb_index] = 1
    return lr.predict([x])[0]

In [ ]:
price_prediction('Obręb_3-01-05', 2, 90, 3)